In [8]:
import pandas as pd



In [9]:
print("Hello World")

Hello World


In [10]:
df = pd.read_csv("transactions.csv")
low_memory=False
df.isnull().sum()

C:\Users\Max\AppData\Local\Temp\ipykernel_11400\522436634.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("transactions.csv")


transaction_id              0
timestamp                   0
amount                      0
currency                    0
sender_account              0
receiver_account            0
sender_country            500
sender_municipality       500
receiver_country          500
receiver_municipality     500
transaction_type            0
notes                    9982
dtype: int64

In [11]:
print(df.duplicated())

0        False
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Length: 100000, dtype: bool


In [12]:
new_df = df.drop_duplicates(inplace = True)

In [23]:
def clean_amount(value):
    if pd.isna(value):
        return None

    value = str(value).replace(" ", "").replace(",", ".")
    try:
        return float(value)
    except ValueError:
        return None

In [13]:
df["amount"] = df["amount"].apply(clean_amount)

NameError: name 'clean_amount' is not defined

In [14]:
high_amount = df[df["amount"] > 100000]
print(high_amount)

TypeError: '>' not supported between instances of 'str' and 'int'

In [15]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', format="%Y%m%d %H:%M:%S")
df_sorted = df.sort_values(['sender_account', 'timestamp'])
df_sorted['time_diff'] = df_sorted.groupby('sender_account')['timestamp'].diff().dt.total_seconds()
rapid_transfers = df_sorted[df_sorted['time_diff'] < 60]

print(rapid_transfers)

Empty DataFrame
Columns: [transaction_id, timestamp, amount, currency, sender_account, receiver_account, sender_country, sender_municipality, receiver_country, receiver_municipality, transaction_type, notes, time_diff]
Index: []


In [16]:
df['hour'] = df['timestamp'].dt.hour
odd_hours = df[(df['hour'] >= 0) & (df['hour'] <= 5)]

print(odd_hours)

                           transaction_id           timestamp    amount  \
372  e13b1664-c549-4f92-895a-1f941dd6d82e 2025-01-25 04:48:00  12881.63   

    currency            sender_account          receiver_account  \
372      SEK  SE8902IOSH72112913547339  SE8902EXDO77351409949930   

    sender_country sender_municipality receiver_country receiver_municipality  \
372         Sweden          Norrköping           Sweden            Trelleborg   

    transaction_type           notes  hour  
372         incoming  Loan repayment   4.0  


In [17]:
df[df["currency"] == "USD"]

,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,hour
10024,9dc886e6-92b4-4676-a117-898a49cd7028,NaT,33122.79,USD,GB62RKHP79010933767921,SE8902HQAX24109353987935,Zimbabwe,Barrport,Sweden,Nyköping,incoming,Tax refund,NaN
10027,8e9b3ce5-7fbb-4bcf-9972-e7b94b746024,NaT,36723.54,USD,GB15VDIW08275286495785,SE8902IKDV65851419590384,Estonia,Holmfurt,Sweden,Uppsala,incoming,Utility bill,NaN
10038,44649832-79ae-4434-b56b-85724eae71fe,NaT,18030.9,USD,GB30KWRY94445383264469,SE8902KYVM99796065752709,Ecuador,Nancyville,Sweden,Piteå,incoming,Subscription fee,NaN
10042,2f20e79c-557d-4373-8927-ffc5171be76e,NaT,47793.06,USD,GB38KUZO58009177664053,SE8902BGXZ28785412485874,Lebanon,Morrismouth,Sweden,Borås,outgoing,Consulting fee,NaN
10046,7f24aabc-260f-410e-9258-e4d45d61ae4f,NaT,25048.93,USD,GB39ORSX60465275895880,SE8902FGVT78270418758930,Zimbabwe,Port Chrisland,Sweden,Lund,outgoing,Subscription fee,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,ea7e0cb2-fadb-496c-962b-7e54a5cfa1bc,NaT,146.04,USD,GB39XHRG15445191862019,SE8902AQYT55466061009593,Mauritania,Jacksonville,Sweden,Nyköping,outgoing,Gift to family member,NaN
99996,4271c3c0-d3be-49f5-8154-ee60e2a1e582,NaT,56.98,USD,GB97OHYP64183530899251,SE8902AQYT55466061009593,Guinea,Waltersport,Sweden,Skellefteå,incoming,NaN,NaN
99997,23574238-9739-4af4-a823-5b699812db4d,NaT,128.34,USD,GB68PWPH34390331224741,SE8902AQYT55466061009593,Tokelau,Davidfurt,Sweden,Luleå,incoming,Gift to family member,NaN
99998,38e785ff-e460-419e-af07-595f5c67aaf6,NaT,149.13,USD,GB30NDUZ96025046626638,SE8902AQYT55466061009593,Bulgaria,West Andrewside,Sweden,Skellefteå,outgoing,Loan repayment,NaN


In [18]:
df["currency"].unique()

array(['SEK', 'SKR', ' SEK', 'SEK ', 'DKK', 'USD', 'EUR', 'NOK', 'RMB',
       'ZAR', 'GBP', 'ZMW', 'JPY'], dtype=object)

In [19]:
def normalize_currency(code):
    #value = value.replace(" ", "").replace(",", ".")
    code = code.upper()
    return "CNY" if code == "RMB" else code

In [20]:
df["currency"] = df["currency"].apply(normalize_currency)
df[df["currency"] == "CNY"]

,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,hour
10388,8587f935-217f-4f0f-bf70-f49d71fc7560,NaT,29504.95,CNY,GB22CUIS74011346478461,SE8902FVHN39495333689229,Myanmar,North Geraldine,Sweden,Trelleborg,outgoing,Online purchase,NaN
10869,dac2b381-8e65-4354-8273-e37bc9844c3d,NaT,46625.75,CNY,GB60QSLL17949114419711,SE8902JGET69909203043484,China,磊市,Sweden,Ängelholm,outgoing,Utility bill,NaN
11412,b9f1fcf0-d6c3-43c7-a41c-d450bc7108e1,NaT,6779.96,CNY,GB87KCWK96719669434369,SE8902CUOE57010683673827,China,兴城市,Sweden,Umeå,incoming,NaN,NaN
12473,55f24179-c080-444f-bb19-70a69c8ac71d,NaT,24002.99,CNY,GB12FDWJ38091332215841,SE8902ZDKY31012668904698,China,关岭市,Sweden,Östersund,incoming,NaN,NaN
12835,0cca4ee5-11e1-4dff-8bd1-d0d22543f199,NaT,29706.17,CNY,GB83JZAR87406000684066,SE8902ELZB18739033043345,Myanmar,Barlowland,Sweden,Sandviken,incoming,Subscription fee,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99723,e91aff2a-7c51-4397-8287-4b7f6929a115,NaT,8.78,CNY,GB67FMYV42874675055666,SE8902NKQD67441527476901,Myanmar,Carolfurt,Sweden,Gävle,outgoing,Online purchase,NaN
99748,b14d5379-e763-4a9a-981f-bb016c7407c4,NaT,120.24,CNY,GB29IJPZ79806661694971,SE8902ALDV54212317929105,Myanmar,Rossstad,Sweden,Skövde,outgoing,Rent transfer,NaN
99774,9b719925-06a1-49f1-b228-94bca1021bde,NaT,126.68,CNY,GB68VJWO80137508247615,SE8902TLVN08996057650298,China,永安县,Sweden,Karlstad,incoming,Subscription fee,NaN
99821,a09c2543-2611-49da-9557-90196009f5e7,NaT,175.47,CNY,GB75IMUT16063944814662,SE8902ZXAW60732336695039,China,辛集县,Sweden,Motala,outgoing,NaN,NaN


In [21]:
df['receiver_account'].value_counts()[lambda x: x > 1]

receiver_account
SE8902ALDV54212317929105    78
SE8902XNNG22633901157104    78
SE8902FMJT85003375468035    77
SE8902PZOD95284482487556    77
SE8902MEUJ12583052373274    75
                            ..
SE8902TIKJ28529081186459    34
SE8902GPEX80518454024672    33
SE8902MHLN19630671655118    33
SE8902WQNH22089611973896    30
SE8902UDDT61255542548403    30
Name: count, Length: 1000, dtype: int64

In [22]:
multiple_received = df.groupby('receiver_account').size()
multiple_received = multiple_received[multiple_received > 75]
print(multiple_received)

receiver_account
SE8902ALDV54212317929105    78
SE8902FMJT85003375468035    77
SE8902PZOD95284482487556    77
SE8902XNNG22633901157104    78
dtype: int64


In [23]:
# Se till att amount är i rätt format (float)
#df['amount'] = df['amount'].astype(float)

# Grupp efter sender och receiver
transfer_summary = df.groupby(['sender_account', 'receiver_account']).agg(
    transaction_count=('amount', 'count'),
    #total_amount=('amount', 'sum')
).reset_index()

# Visa bara de som skickat fler än en gång
repeated_transfers = transfer_summary[transfer_summary['transaction_count'] > 1]

print(repeated_transfers)


                 sender_account          receiver_account  transaction_count
42709  SE8902ABCI09083135852362  SE8902IADF98077782900234                  2
49579  SE8902CWDO20077844008455  SE8902WMJJ51991398288050                  2
53852  SE8902EXIB33653688394695  SE8902SEOH55592315458862                  2
54334  SE8902FCQC45454489492127  SE8902AXJS46097747161697                  2
56606  SE8902GGKR42392585018307  SE8902PAUD14800167956656                  2
58182  SE8902GXXP17738713863102  SE8902LHFZ29797803618611                  2
61010  SE8902IFTV45914273013389  SE8902TRCO62989417471868                  2
62189  SE8902IRAV26677525304054  SE8902KXIU12156687445452                  2
63343  SE8902JGET69909203043484  SE8902CZTD12546658942322                  2
63346  SE8902JGET69909203043484  SE8902TCVN64620007776552                  2
63706  SE8902JMYP35839764449656  SE8902PNMZ47985736012731                  2
66755  SE8902KXQZ51570419970356  SE8902YRMM27818499157642                  2